<a href="https://colab.research.google.com/github/anna-boser/AutoGluon_PM_paper/blob/main/code/ML_models_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Uninstall mkl for faster neural-network training time
!pip uninstall -y mkl
# Upgrade pip to ensure the latest package versions are available
!pip install -U pip
# Upgrade setuptools to be compatible with namespace packages
!pip install -U setuptools
!pip install -U "mxnet<2.0.0"
# Install pre-release, frozen to a particual pre-release for stability
!pip install --pre "autogluon==0.0.16b20201214"
!pip install -U ipykernel

Uninstalling mkl-2019.0:
  Successfully uninstalled mkl-2019.0
     |████████████████████████████████| 1.6MB 8.5MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 46.9 MB 62 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 309 kB 11.9 MB/s 
     |████████████████████████████████| 242 kB 14.2 MB/s 
     |████████████████████████████████| 41 kB 433 kB/s 
     |████████████████████████████████| 964 kB 14.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 131 kB 52.2 MB/s 
     |████████████████████████████████| 101 kB 6.7 MB/s 
     |████████████████████████████████| 206 kB 14.0 MB/s 
     |██████████████

     |████████████████████████████████| 120 kB 7.5 MB/s 
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 5.5.4 which is incompatible.


In [ ]:
import autogluon as ag
from autogluon.tabular import TabularPrediction as task
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import gc
import math
import os
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/NASA PM_estimation/')

Mounted at /content/drive


In [ ]:
small_grid = pd.read_csv("Data/datasets/Small_Grid.csv")
df = pd.read_csv("Data/datasets/Train.csv")

In [ ]:
# Make a dataset with the average PM by day
PM = df[["Day", "PM"]].groupby('Day').mean().reset_index()
# print(PM.head())

medianPM_day = np.array(PM[PM['PM']==np.median(PM['PM'])]['Day'])
maxPM_day = np.array(PM[PM['PM']==np.amax(PM['PM'])]['Day'])

In [ ]:
def fitgrid(hyperparameters):
  features = ['Day', 'Elevation', 'Emissions', 'Forest',
      'Roads', 'Streets', 'Plumes_High', 'Plumes_Med', 'Plumes_Low',
      'Max_Temp', 'Max_Wind', 'Precip', 'Rel_Humidity', 'Wind_Dir', 'BLH',
      'AOD', 'PM']
  label_column = 'PM'

  train_data = task.Dataset(df[features])
  test_data = task.Dataset(small_grid[features])

  predictor = task.fit(train_data=train_data, 
                        label=label_column, 
                        hyperparameters=hyperparameters, 
                        # time_limits = 60*3,
                        # hyperparameter_tune=True, 
                        eval_metric='r2')

  test_data_nolab = test_data.drop(labels = [label_column], axis = 1)
  # y_pred = predictor.predict(test_data_nolab)
  test_data['y_pred'] = predictor.predict(test_data_nolab)
  print(test_data.head())
  test_data['Id'] = small_grid['Id']
  test_data['Day'] = test_data['Day'].apply(lambda x: 'Day'+str(x))
  test_data = test_data.pivot(index='Id', columns='Day', values='y_pred')

  return test_data

In [ ]:
CAT = fitgrid({'CAT': {}})
CAT.to_csv("Data/output/grids/CAT.csv", index = True)
CAT.mean(axis = 1).to_csv("Data/output/grids/avg/CAT_Avg.csv", index = True)
CAT[['Day'+str(maxPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/high/CAT_high.csv", index = True)
CAT[['Day'+str(medianPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/low/CAT_low.csv", index = True)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_151200/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_151200/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9900
Train Data Columns: 16
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.81719, 7.72732)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11800.06 MB
	Train Data (Original)  Memory Usage: 1.27 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to 

   Day  Elevation  Emissions  Forest  ...         BLH  AOD  PM     y_pred
0    1        0.0        0.0       0  ...  978.736816  NaN NaN   9.099906
1   10        0.0        0.0       0  ...  432.832642  NaN NaN   4.846766
2  100        0.0        0.0       0  ...   45.136700  NaN NaN   7.670850
3  101        0.0        0.0       0  ...  147.480194  NaN NaN  10.681348
4  102        0.0        0.0       0  ...  248.710907  NaN NaN   5.391977

[5 rows x 18 columns]


In [ ]:
NN = fitgrid({'NN': {}})
NN.to_csv("Data/output/grids/NN.csv", index = True)
NN.mean(axis = 1).to_csv("Data/output/grids/avg/NN_Avg.csv", index = True)
NN[['Day'+str(maxPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/high/NN_high.csv", index = True)
NN[['Day'+str(medianPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/low/NN_low.csv", index = True)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_154654/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_154654/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9900
Train Data Columns: 16
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.81719, 7.72732)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11576.96 MB
	Train Data (Original)  Memory Usage: 1.27 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to 

   Day  Elevation  Emissions  Forest  ...         BLH  AOD  PM     y_pred
0    1        0.0        0.0       0  ...  978.736816  NaN NaN   8.955448
1   10        0.0        0.0       0  ...  432.832642  NaN NaN  12.338755
2  100        0.0        0.0       0  ...   45.136700  NaN NaN  10.635022
3  101        0.0        0.0       0  ...  147.480194  NaN NaN  18.607294
4  102        0.0        0.0       0  ...  248.710907  NaN NaN  14.622022

[5 rows x 18 columns]


In [ ]:
GBM = fitgrid({'GBM': {}})
GBM.to_csv("Data/output/grids/GMB.csv", index = True)
GBM.mean(axis = 1).to_csv("Data/output/grids/avg/GMB_Avg.csv", index = True)
GBM[['Day'+str(maxPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/high/GMB_high.csv", index = True)
GBM[['Day'+str(medianPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/low/GMB_low.csv", index = True)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_155100/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_155100/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9900
Train Data Columns: 16
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.81719, 7.72732)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11546.24 MB
	Train Data (Original)  Memory Usage: 1.27 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to 

[1000]	train_set's l2: 0.757154	train_set's r2: 0.986108	valid_set's l2: 8.87936	valid_set's r2: 0.813689


	0.8153	 = Validation r2 score
	5.49s	 = Training runtime
	0.12s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.8153	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 6.35s ...


   Day  Elevation  Emissions  Forest  ...         BLH  AOD  PM     y_pred
0    1        0.0        0.0       0  ...  978.736816  NaN NaN   8.286654
1   10        0.0        0.0       0  ...  432.832642  NaN NaN   3.303148
2  100        0.0        0.0       0  ...   45.136700  NaN NaN   8.712801
3  101        0.0        0.0       0  ...  147.480194  NaN NaN  11.388944
4  102        0.0        0.0       0  ...  248.710907  NaN NaN   4.925733

[5 rows x 18 columns]


In [ ]:
RF = fitgrid({'RF': {}})
RF.to_csv("Data/output/grids/RF.csv", index = True)
RF.mean(axis = 1).to_csv("Data/output/grids/avg/RF_Avg.csv", index = True)
RF[['Day'+str(maxPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/high/RF_high.csv", index = True)
RF[['Day'+str(medianPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/low/RF_low.csv", index = True)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_182238/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_182238/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9900
Train Data Columns: 16
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.81719, 7.72732)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11814.51 MB
	Train Data (Original)  Memory Usage: 1.27 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to 

   Day  Elevation  Emissions  Forest  ...         BLH  AOD  PM     y_pred
0    1        0.0        0.0       0  ...  978.736816  NaN NaN  10.218667
1   10        0.0        0.0       0  ...  432.832642  NaN NaN   4.897666
2  100        0.0        0.0       0  ...   45.136700  NaN NaN  10.132334
3  101        0.0        0.0       0  ...  147.480194  NaN NaN  10.465777
4  102        0.0        0.0       0  ...  248.710907  NaN NaN   6.422389

[5 rows x 18 columns]


In [ ]:
XT = fitgrid({'XT': {}})
XT.to_csv("Data/output/grids/XT.csv", index = True)
XT.mean(axis = 1).to_csv("Data/output/grids/avg/XT_Avg.csv", index = True)
XT[['Day'+str(maxPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/high/XT_high.csv", index = True)
XT[['Day'+str(medianPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/low/XT_low.csv", index = True)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_182600/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_182600/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9900
Train Data Columns: 16
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.81719, 7.72732)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11811.17 MB
	Train Data (Original)  Memory Usage: 1.27 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to 

   Day  Elevation  Emissions  Forest  ...         BLH  AOD  PM    y_pred
0    1        0.0        0.0       0  ...  978.736816  NaN NaN  6.907166
1   10        0.0        0.0       0  ...  432.832642  NaN NaN  5.191667
2  100        0.0        0.0       0  ...   45.136700  NaN NaN  9.248000
3  101        0.0        0.0       0  ...  147.480194  NaN NaN  9.151000
4  102        0.0        0.0       0  ...  248.710907  NaN NaN  6.492833

[5 rows x 18 columns]


In [ ]:
KNN = fitgrid({'KNN': {}})
KNN.to_csv("Data/output/grids/KNN.csv", index = True)
KNN.mean(axis = 1).to_csv("Data/output/grids/avg/KNN_Avg.csv", index = True)
KNN[['Day'+str(maxPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/high/KNN_high.csv", index = True)
KNN[['Day'+str(medianPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/low/KNN_low.csv", index = True)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_182834/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_182834/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9900
Train Data Columns: 16
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.81719, 7.72732)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11796.93 MB
	Train Data (Original)  Memory Usage: 1.27 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to 

   Day  Elevation  Emissions  Forest  ...         BLH  AOD  PM  y_pred
0    1        0.0        0.0       0  ...  978.736816  NaN NaN    7.03
1   10        0.0        0.0       0  ...  432.832642  NaN NaN    8.48
2  100        0.0        0.0       0  ...   45.136700  NaN NaN   11.90
3  101        0.0        0.0       0  ...  147.480194  NaN NaN    8.70
4  102        0.0        0.0       0  ...  248.710907  NaN NaN    6.47

[5 rows x 18 columns]


In [ ]:
Default = fitgrid('default')
Default.to_csv("Data/output/grids/Default.csv", index = True)
Default.mean(axis = 1).to_csv("Data/output/grids/avg/Default_Avg.csv", index = True)
Default[['Day'+str(maxPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/high/Default_high.csv", index = True)
Default[['Day'+str(medianPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/low/Default_low.csv", index = True)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_183033/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_183033/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9900
Train Data Columns: 16
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.81719, 7.72732)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11787.01 MB
	Train Data (Original)  Memory Usage: 1.27 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to 

[1000]	train_set's l2: 0.757154	train_set's r2: 0.986108	valid_set's l2: 8.87936	valid_set's r2: 0.813689


	0.8153	 = Validation r2 score
	5.37s	 = Training runtime
	0.12s	 = Validation runtime
Fitting model: LightGBMXT ...


[1000]	train_set's l2: 3.68413	train_set's r2: 0.938201	valid_set's l2: 10.0657	valid_set's r2: 0.789473


	0.7921	 = Validation r2 score
	5.45s	 = Training runtime
	0.12s	 = Validation runtime
Fitting model: CatBoost ...
	0.8062	 = Validation r2 score
	11.31s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: XGBoost ...
	0.8571	 = Validation r2 score
	16.36s	 = Training runtime
	0.09s	 = Validation runtime
Fitting model: NeuralNetMXNet ...
	0.6128	 = Validation r2 score
	65.08s	 = Training runtime
	0.16s	 = Validation runtime
Fitting model: LightGBMCustom ...


[1000]	train_set's l2: 0.150177	train_set's r2: 0.996147	valid_set's l2: 12.1705	valid_set's r2: 0.74503


	0.7487	 = Validation r2 score
	13.45s	 = Training runtime
	0.19s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.8581	 = Validation r2 score
	0.53s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 150.78s ...


   Day  Elevation  Emissions  Forest  ...         BLH  AOD  PM     y_pred
0    1        0.0        0.0       0  ...  978.736816  NaN NaN   8.329772
1   10        0.0        0.0       0  ...  432.832642  NaN NaN   3.859710
2  100        0.0        0.0       0  ...   45.136700  NaN NaN   8.874905
3  101        0.0        0.0       0  ...  147.480194  NaN NaN  12.082391
4  102        0.0        0.0       0  ...  248.710907  NaN NaN   6.324549

[5 rows x 18 columns]


In [ ]:
TwoStage = pd.read_csv("Data/output/grids/2S_Full_Grid.csv", index_col = 0)
TwoStage.mean(axis = 1).to_csv("Data/output/grids/avg/TwoStage_Avg.csv", index = True)
TwoStage[['Day'+str(maxPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/high/TwoStage_high.csv", index = True)
TwoStage[['Day'+str(medianPM_day[0])]].mean(axis = 1).to_csv("Data/output/grids/low/TwoStage_low.csv", index = True)